In [2]:
import pandas as pd

from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report

import xgboost as xgb
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

GBDT = GradientBoostingClassifier()

'''
分类器及参数如下：
KNN ：k=30
RF : 默认参数
GBDT: 默认参数
XGboost : 默认  xgb.XGBClassifier()
SVC : 默认

'''
SVC()

classifier1Set = [xgb.XGBClassifier(), RandomForestClassifier(), GradientBoostingClassifier(), ]
classifier2Set = []


reports = {}

# 加载数据集
df_loan = pd.read_csv("/Users/mxfmeng/PycharmProjects/GraduationDesign/MLmodels/dataset/loan_new_3_1.csv")
df_loan.drop(columns=['loan_status'], inplace=True)

# 设置分类器1的target : 按label分类，无风险（0）和有风险（1、2、3）分成两类
df_loan['risk'] = df_loan['label'].apply(lambda row: 0 if row == 0 else 1)

# 划分 训练集 和 测试集
X_train, X_finalTest, y_train, y_finalTest = train_test_split(df_loan, df_loan[['risk']], test_size=0.2,
                                                              random_state=0)

''' 分类器1：划分"无风险"和"有风险" '''
# 对classifier1的训练集再次划分测试集，用于单独统计正确率
X1_train, X1_test, y1_train, y1_test = train_test_split(X_train.drop(columns=['label', 'risk']), y_train, test_size=0.2,
                                                        random_state=0)
classifier1 = xgb.XGBClassifier()
classifier1.fit(X1_train, y1_train)
Y1_pred = classifier1.predict(X1_test.values)
# 保存结果1
reports['classifier1'] = classification_report(y1_test, Y1_pred)

''' 分类器2：将"有风险"的进一步划分成"低"、"中"、"高" '''
# 筛选出classifier2的训练数据（risk=1）
X2 = X_train[X_train['risk'] == 1]
y2 = X2['label'] - 1  # {1,2,3} 不合法，转换为{0,1,2}
# 对classifier2的训练集再次划分测试集，用于单独统计正确率
X2_train, X2_test, y2_train, y2_test = train_test_split(X2.drop(columns=['label', 'risk']), y2, test_size=0.2,
                                                        random_state=0)
classifier2 = xgb.XGBClassifier()
classifier2.fit(X2_train, y2_train)
Y2_pred = classifier2.predict(X2_test.values)
# 保存结果2
reports['classifier2'] = classification_report(y2_test, Y2_pred)

''' 连接两个classifier: 用之前划分的测试集 X_finalTest 和 y_finalTest ,测试最终分类效果 '''
y_finalTest = X_finalTest['label']

c1_input = X_finalTest.drop(columns=['risk', 'label'])
c1_output = classifier1.predict(c1_input.values)  # c1_output 表示数据是否有风险(0,1)
# 按预测结果筛选出 classifier2 的 input
X_finalTest['pred'] = c1_output
c2_input = X_finalTest[X_finalTest['pred'] == 1]
c2_input.drop(columns=['label', 'risk', 'pred'], inplace=True)
# classifier2 预测分类结果
c2_output = classifier2.predict(c2_input.values)


# classifier2 结果
finalRes = pd.DataFrame()
finalRes['result'] = c2_output + 1
finalRes['index'] = c2_input.index
# classifier1 结果
tmp = pd.DataFrame()
tmp['res'] = c1_output
tmp['index'] = X_finalTest.index
tmp = tmp[tmp['res'] == 0]
tmp.rename(columns={'res': 'result'}, inplace=True)
# 合并结果
finalRes = pd.concat([finalRes, tmp], ignore_index=True)
finalRes.sort_values(by='index', inplace=True)
# 统计准确率
y_finalTest.sort_index(inplace=True)
reports['final'] = classification_report(y_finalTest, finalRes['result'])


print('\nclassifier1:\n', reports['classifier1'])
print('\nclassifier2:\n', reports['classifier2'])
print('\nfinal:\n', reports['final'])


/var/folders/8m/6cpmqrns19n25q6dpm3wnyzh0000gn/T/ipykernel_26820/2846668779.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  c2_input.drop(columns=['label', 'risk', 'pred'], inplace=True)



classifier1:
               precision    recall  f1-score   support

           0       0.98      0.98      0.98     25947
           1       0.99      0.99      0.99    104729

    accuracy                           0.99    130676
   macro avg       0.99      0.99      0.99    130676
weighted avg       0.99      0.99      0.99    130676


classifier2:
               precision    recall  f1-score   support

           0       0.94      1.00      0.97     95603
           1       0.17      0.00      0.00      1588
           2       0.93      0.39      0.55      7504

    accuracy                           0.94    104695
   macro avg       0.68      0.46      0.51    104695
weighted avg       0.93      0.94      0.92    104695


final:
               precision    recall  f1-score   support

           0       0.98      0.97      0.98     32492
           1       0.94      0.99      0.96    119626
           2       0.12      0.00      0.00      1968
           3       0.93      0.39   

In [3]:
# classifier2 结果
finalRes = pd.DataFrame()
finalRes['result'] = c2_output + 1
finalRes['index'] = c2_input.index
finalRes

,result,index
0,1,254394
1,1,124943
2,1,330472
3,1,590460
4,1,492812
...,...,...
130987,1,363950
130988,1,214994
130989,1,711135
130990,1,689178


In [4]:
tmp = pd.DataFrame()
tmp['res'] = c1_output
tmp['index'] = X_finalTest.index
tmp = tmp[tmp['res'] == 0]
tmp.rename(columns={'res': 'result'}, inplace=True)
tmp

,result,index
8,0,664434
10,0,400505
11,0,333775
12,0,761990
13,0,5726
...,...,...
163306,0,280501
163307,0,95735
163314,0,198116
163324,0,83751


In [5]:
# 合并结果
finalRes = pd.concat([finalRes, tmp], ignore_index=True)
finalRes.sort_values(by='index', inplace=True)
finalRes

,result,index
59691,1,6
65901,1,7
151629,0,19
115641,1,31
61526,1,36
...,...,...
149998,0,816690
24767,1,816697
124653,1,816702
128957,1,816710


In [7]:
y_finalTest

6         1
7         1
19        0
31        1
36        1
         ..
816690    0
816697    1
816702    1
816710    1
816714    3
Name: label, Length: 163345, dtype: int64